In [1]:
import os
import pickle

from features import *

In [4]:
with open('model' , 'rb') as f:
    xgb = pickle.load(f)

In [2]:
with open('model_dict.pickle' , 'rb') as f:
    model = pickle.load(f)

In [3]:
model

{'bs': RandomForestRegressor(max_depth=20, max_features='sqrt', n_estimators=2000,
                       n_jobs=-1, random_state=7531),
 'h': RandomForestRegressor(max_depth=20, max_features='sqrt', n_estimators=2000,
                       n_jobs=-1, random_state=7531),
 'hm': RandomForestRegressor(max_depth=20, max_features='sqrt', n_estimators=2000,
                       n_jobs=-1, random_state=7531),
 'ol': RandomForestRegressor(max_depth=20, max_features='sqrt', n_estimators=2000,
                       n_jobs=-1, random_state=7531),
 'del': RandomForestRegressor(max_depth=20, max_features='sqrt', n_estimators=2000,
                       n_jobs=-1, random_state=7531),
 'el': RandomForestRegressor(max_depth=20, max_features='sqrt', n_estimators=2000,
                       n_jobs=-1, random_state=7531),
 'os': RandomForestRegressor(max_depth=20, max_features='sqrt', n_estimators=2000,
                       n_jobs=-1, random_state=7531),
 'wob': RandomForestRegressor(max_depth=2

In [12]:
data_dir = '/hkfs/work/workspace/scratch/bh6321-energy_challenge/data/'
save_dir = 'forecasts/'

In [13]:
# dataloader
test_file = os.path.join(data_dir, 'test.csv')
valid_file = os.path.join(data_dir, 'valid.csv')

data_file = test_file if os.path.exists(test_file) else valid_file

In [14]:
data_file

'/hkfs/work/workspace/scratch/bh6321-energy_challenge/data/valid.csv'

In [58]:
pd.options.mode.chained_assignment = None

In [54]:
df = pd.read_csv(data_file)

In [55]:
df = preprocess(df)
df = add_time_features(df, drop = True)

In [56]:
df.drop(columns='day_name', inplace=True)

In [57]:
# CATEGORICAL_VARIABLES = ['day_name']

# # one-hot-encoding of categorical features
# df = pd.get_dummies(df, columns = CATEGORICAL_VARIABLES)

In [59]:
df_list = add_ts_features(df, return_as_list=True)

In [60]:
forecast_dict = dict()
for df_tmp in df_list:
    city = df_tmp.city.unique()[0]
    print(city)
    df_tmp.drop(columns='city', inplace=True)
    pred = model[city].predict(df_tmp)
    forecast_dict[city] = pred

bs
h
hm
ol
del
el
os
wob
hi
whv
lg
ce
go
sz


In [44]:
forecast_dict

{'bs': array([219.00212969, 213.40765118, 204.07711395, ..., 261.65559443,
        267.37731205, 261.99892601]),
 'h': array([595.72750848, 585.85908414, 595.569256  , ..., 733.79838856,
        696.25317395, 658.14632712]),
 'hm': array([189.88723749, 187.99381489, 188.28669213, ..., 213.74766164,
        208.63585749, 205.34701821]),
 'ol': array([163.02161994, 161.18861286, 155.77087321, ..., 178.46701189,
        161.67751038, 153.68273351]),
 'del': array([124.71822539, 124.66486443, 125.99687123, ..., 149.48330114,
        143.24965064, 139.8664225 ]),
 'el': array([69.65273544, 64.93499997, 61.78841001, ..., 82.21574408,
        75.16569173, 73.23667747]),
 'os': array([59.41120453, 56.491366  , 56.0216138 , ..., 84.59179604,
        75.11173097, 69.56404197]),
 'wob': array([64.17896413, 61.8643449 , 60.81880193, ..., 78.73077106,
        73.28360566, 67.95784076]),
 'hi': array([52.76998666, 49.30199409, 47.25557991, ..., 65.77820702,
        63.67671023, 58.50170777]),
 'whv'

In [64]:
y_pred = np.array([])

In [68]:
for f in forecast_dict.values():
    y_pred = np.concatenate([y_pred, f[24*7:-24*7]])

In [74]:
df.shape

(122640, 17)

In [75]:
730*14

10220

In [70]:
len(y_pred)

117936

In [73]:
len(forecast_dict['bs'])

8760

In [76]:
len(forecast_dict['bs']) -14*24

8424

In [72]:
117936/14

8424.0

In [52]:
8424*14

117936

In [50]:
df.shape

(122640, 17)

In [ ]:
y_pred = xgb.predict(X)

In [21]:
X.shape

(122640, 36)

In [19]:
y_pred.shape

(122640,)

In [13]:
submission = pd.DataFrame(np.reshape(y_pred, (len(y_pred)//(24*7), 24*7)))

In [14]:
submission.shape

(730, 168)

In [17]:
# save to csv
result_path = os.path.join(save_dir, 'forecasts.csv')
submission.to_csv(result_path, header=False, index=False)

print(f"Done! The result is saved in {result_path}")

Done! The result is saved in forecasts/forecasts.csv
